In [4]:
!git clone https://github.com/abewley/sort.git


Cloning into 'sort'...


In [ ]:
%pip install -q ultralytics opencv-python filterpy
%pip install -q scikit-image


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\HP\AppData\Local\Programs\Python\Python313\python.exe -m pip install [options] <archive url/path> ...

no such option: --quet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import sys
sys.path.append("sort")
from sort import Sort  # SORT tracker
import matplotlib.pyplot as plt
import os

# 2. Load video
video_path = "../utilities/cars_on_the_highway.mp4"
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found. Please check the path.")

cap = cv2.VideoCapture(video_path)

# 3. Load YOLO model for detection
model_path = "../utilities/yolov8n.pt"  # Path to your YOLO model
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file {model_path} not found. Please check the path.")

model = YOLO(model_path)

# 4. Inicialize SORT tracker
tracker = Sort()

# 5. Storing tracking data
tracked_data = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, classes=[2, 5, 7], conf=0.4, verbose=False)  # car, bus, truck
    detections = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().item()
            detections.append([x1, y1, x2, y2, conf])

    det_array = np.array(detections)

    if det_array.shape[0] == 0:
        det_array = np.empty((0, 5))  # Empty array if no detections

    tracks = tracker.update(det_array)


    for track in tracks:
        x1, y1, x2, y2, track_id = track.astype(int)
        tracked_data.append([frame_idx, track_id, x1, y1, x2, y2])
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    frame_idx += 1
    # cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
    # cv2.resizeWindow("Tracking", 800, 600)
    # cv2.imshow("Tracking", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break


cap.release()
cv2.destroyAllWindows()

# 6. Saving data to .csv file
columns = ["frame", "id", "x1", "y1", "x2", "y2"]
df = pd.DataFrame(tracked_data, columns=columns)
df.to_csv("trajectory_data_sort.csv", index=False)
print("Tracking data saved to trajectory_data_sort.csv")


Tracking data saved to trajectory_data_sort.csv
